# Project 3 - Collaboration and Competition

---

This notebook implements the solution for project 3 of Udacity's Deep Reinforcement Learning Degree

### 1. Tennis - Multi-agent Environment

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

The task is episodic, and in order to solve the environment, your agents must get an average score of +0.5 (over 100 consecutive episodes, after taking the maximum over both agents). Specifically,

After each episode, we add up the rewards that each agent received (without discounting), to get a score for each agent. This yields 2 (potentially different) scores. We then take the maximum of these 2 scores.
This yields a single score for each episode.
The environment is considered solved, when the average (over 100 episodes) of those scores is at least +0.5.

In [ ]:
import torch
import numpy as np
from unityagents import UnityEnvironment
from collections import namedtuple, deque

In [ ]:
from agent import Agent

In [ ]:
env = UnityEnvironment(file_name="envs/Tennis.app", no_graphics=True)

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. (Optional) Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. Solving the environment

In [ ]:
# Reset the environment
env_info = env.reset(train_mode=True)[brain_name]

In [ ]:
# Initialize our agent
agent = Agent(state_size, action_size, seed=42)

In [ ]:
def ma_ddpg(n_episodes=20000, print_every=100, stop_score=0.5, early_stop=True):
    """DDQN Algorithm.
    
    Params
    ======
        n_episodes (int): Max number of training episodes
        max_t (int): Max number of timesteps per episode in case you want to stop early (<1000)
        print_every (int): How often should we print the info
        stop_score (int): Target score to stop the training process
    """
    num_agents = 2
    
    scores_deque = deque(maxlen=100)
    scores = []
    avg_scores = []
    max_score = 0

    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        states = env_info.vector_observations                 # get the current state (for each agent)
        score = np.zeros(num_agents)                          # initialize the score (for each agent)

        while True:
            
            actions = [agent.act(state.reshape([1,state.shape[0]])) for state in states]
            
            env_info = env.step(actions,)[brain_name]
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished 
            
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done, i_episode,)
            states = next_states
            score += rewards                         
            
            if np.any(dones):
                break
        
        scores_deque.append(np.max(score))
        scores.append(np.max(score))
        avg_scores.append(np.mean(scores_deque))
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")

        
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            if np.mean(scores_deque) > max_score:
                torch.save(agent.actor_local.state_dict(), 'trained_models/checkpoint_actor.pth')
                torch.save(agent.critic_local.state_dict(), 'trained_models/checkpoint_critic.pth')
                max_score = np.mean(scores_deque)
        
        if np.mean(scores_deque) >= stop_score and early_stop:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), 'trained_models/checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'trained_models/checkpoint_critic.pth')
            break        
                    
    return scores, avg_scores

In [ ]:
# Run the training job.
# This might take a while!

scores, avg_scores = ma_ddpg(n_episodes=8000, print_every=1000, early_stop=False)

In [ ]:
env.close()

In [ ]:
import matplotlib.pyplot as plt 
n_episodes = len(scores)

fig, ax = plt.subplots(figsize=(15, 10))
ax.plot(np.linspace(1,n_episodes+1, n_episodes), scores)
ax.plot(np.linspace(1,n_episodes+1, n_episodes), avg_scores)
ax.set_xlabel('Episodes')
ax.set_ylabel('Score per Episode')
ax.set_title('Training progress of model');

fig.tight_layout()
fig.savefig("images/learning_rate.png")

### 5. Show a trained agent
You can now load a previously trained agent and see how it behaves

In [ ]:
env = UnityEnvironment(file_name="envs/Tennis.app")

In [ ]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
action_size = brain.vector_action_space_size

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
num_agents = len(env_info.agents)
state_size = states.shape[1]
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

In [ ]:
agent = Agent(state_size, action_size, seed=42)

In [ ]:
# Load the trained weights
agent.actor_local.load_state_dict(torch.load('trained_models/checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('trained_models/checkpoint_critic.pth'))


In [ ]:
while True:
    actions = agent.act(states)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    dones = env_info.local_done                        # see if episode finished
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break


In [ ]:
env.close()